# 多重パーセプトロン

## 実装の設計
「入力層-隠れ層」、「隠れ層-出力層」の部分は同じ構造となっているので、クラスとして定義できそう。

## Layerクラスの実装
Layerクラスが担うのは層間の結合のため以下の4つを実装する必要がある。

1. \_\_init\_\_ : parametaの初期化
2. forward : 順伝播
3. backward : 逆伝播
4. compute_gradients : モデル学習時の各勾配

1.に関しては「重み、バイアス、活性化関数、活性化関数の微分」の4つを用意する必要がある。この際、活性化関数は変数として設定できるようにしておくのが良い。

2.で層への入力と活性化前の値はそれぞれ勾配計算および誤差逆伝播で用いるために設定している。

3.の逆伝播は2.で定義した活性化前の値を用いて記述する。逆伝播における誤差は活性化関数の微分の変数に活性化前の値を入れるためこのような式となる。

4.は勾配計算である。勾配計算には隠れ層の誤差も必要となるため、引数にその誤差が含まれている。

In [7]:
class Layer():
    #1. __init__ : parametaの初期化
    def __init__(self, input_dim, output_dim, activation, dactivation):
        '''
        インスタンス変数:
        W : 重み
        b : バイアス
        activation : 活性化関数
        dactivation : 活性化関数の微分
        '''
        self.W = np.random.normal(size=(input_dim, output_dim))
        self.b = np.zeros(output_dim)
        self.activation = activation
        self.dactivation = dactivation

    def __call__(self,x):
        return self.forward(x)

    #2. forward : 順伝播
    def forward(self,x):
        self._input = x
        self._pre_activation = np.matmul(x, self.W) + self.b
        return self.activation(self._pre_activation)
    #3. backward : 逆伝播
    def backward(self, delta, W):
        delta = self.dactivation(self._pre_activation) \
                * np.matmul(delta, W.T)
        return delta
    #4. compute_gradients : モデル学習時の各勾配
    def compute_gradients(self, delta):
        dW = np.matmul(self._input.T, delta)
        db = np.matmul(np.ones(self._input.shape[0]), delta)
        return dW, db

## 多層パーセプトロン

In [11]:
import numpy as np
class MLP(object):
    def __init__(self, input_dim, hidden_dim, output_dim):
        '''
        引数:
            input_dim:入力層の次元
            hidden_dim:隠れ層の次元
            output_dim:出力層の次元
        '''
        self.l1 = Layer(input_dim=input_dim,
                        output_dim=hidden_dim,
                        activation=sigmoid,
                        dactivation=dsigmoid)
        self.l2 = Layer(input_dim=hidden_dim,
                        output_dim=output_dim,
                        activation=sigmoid,
                        dactivation=dsigmoid)
        self.layers  = [self.l1, self.l2]
    def __call__(self,x):
        return self.forward(x)
    def forward(self,x):
        h = self.l1(x)
        y = self.l2(h)
        return y

def sigmoid(x):
    return 1/(1+np.exp(-x))

def dsigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

### 1. データの準備
モデルを一般形、すなわち多クラスに対応できるように実装したので、出力tを二次元配列で定義する必要がある
### 2. モデルの構築
### 3. モデルの学習
### 4. モデルの評価

In [16]:
'''
1. データの準備
'''
x = np.array([[0,0], [0,1], [1,0], [1,1]])
t = np.array([[0],[1],[1],[0]])
#t = np.array([0,1,1,0])ではないことに注意
'''
2. モデルの構築
'''
model = MLP(2,2,1)
'''
3. モデルの学習
'''
def compute_loss(t, y):#誤差関数(2クラス版の交差エントロピー誤差)
    return (-t*np.log(y) - (1-t)*np.log(1-y)).sum()
def train_step(x, t):#勾配降下法の」計算。誤差逆伝播は「層を遡る」処理。これを[::-1]で記述。
    y = model(x)
    for i, layer in enumerate(model.layers[::-1]):
        if i == 0:
            delta = y -t
        else:
            delta = layer.backward(delta,W)
        dW, db = layer.compute_gradients(delta)
        layer.W = layer.W - 0.1*dW
        layer.b = layer.b - 0.1*db

        W = layer.W
    loss = compute_loss(t,y)
    return loss

epochs = 10000
for epoch in range(epochs):
    train_loss = train_step(x,t)

    if epoch%1000 ==0 or epoch == epochs -1 :
        print('epoch:{}, loss:{:.3f}'.format(epoch+1, train_loss))

'''
4. モデルの評価
'''
for input in x:
    print('{} => {:.3f}'.format(input, model(input)[0]))

epoch:1, loss:3.441
epoch:1001, loss:0.817
epoch:2001, loss:0.101
epoch:3001, loss:0.050
epoch:4001, loss:0.033
epoch:5001, loss:0.025
epoch:6001, loss:0.020
epoch:7001, loss:0.016
epoch:8001, loss:0.014
epoch:9001, loss:0.012
epoch:10000, loss:0.011
[0 0] => 0.002
[0 1] => 0.997
[1 0] => 0.997
[1 1] => 0.003
